In [137]:
from pyspark.sql.functions import size, udf

In [138]:
from pyspark.sql.types import IntegerType

In [139]:
from pyspark.sql.functions import array_contains

# Load metadata avro

In [140]:
reader = sqlContext.read.format('com.databricks.spark.avro')

In [141]:
meta = reader.load('data/spark_metadata.avro')

In [142]:
meta.columns

['id',
 'path',
 'message_id',
 'date',
 'from',
 'to',
 'cc',
 'bcc',
 'subject',
 'references']

# Loading topic distributions

In [143]:
topdisFile = 'data/enron_small_topic_distributions.tuples'

csvLoader = sqlContext.read.format('com.databricks.spark.csv')
topdis = csvLoader.options(delimiter=',',header='false', inferschema='true').load(topdisFile)

In [144]:
topdis.columns

['C0',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15']

In [145]:
strip_first_col = udf(lambda row: int(row[1:]), IntegerType())
topdis = topdis.withColumn('C0',strip_first_col(topdis['C0']))

# Load dictionary CSV

In [146]:
dicFile = 'enron_small_dic.csv'

csvLoader = sqlContext.read.format('com.databricks.spark.csv')
dic = csvLoader.options(delimiter='\t', header='false', inferschema='true').load(dicFile)
dic = dic.select(dic['C0'].alias('id'), dic['C1'].alias('word'), dic['C2'].alias('count'))

In [147]:
dic.columns

['id', 'word', 'count']

# Load clustertopics CSV

In [148]:
clutoFile = 'enron_small_clustertopics.csv'

csvLoader = sqlContext.read.format('com.databricks.spark.csv')
cluto = csvLoader.options(delimiter=',', header='false', inferschema='true').load(clutoFile)
#dic = dic.select(dic['C0'].alias('id'), dic['C1'].alias('word'), dic['C2'].alias('count'))

In [149]:
cluto.columns

['C0', 'C1', 'C2', 'C3', 'C4']

# Load topicswords CSV

In [150]:
towoFile = 'enron_small_lda_transposed.csv'

csvLoader = sqlContext.read.format('com.databricks.spark.csv')
towo = csvLoader.options(delimiter=',', header='false', inferschema='true').load(towoFile)
#dic = dic.select(dic['C0'].alias('id'), dic['C1'].alias('word'), dic['C2'].alias('count'))'


In [151]:
towo.columns

['C0',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14']

# Merge topdis which has document id and with metadata, based on document id

In [158]:
metasmall = meta.select('id','date')
#topdis_b = topdis.select('C0','C1','C2')
newdf = topdis.join(metasmall, metasmall.id == topdis.C0,'inner')

In [159]:
newdf.columns

['C0',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'id',
 'date']

2236